In [14]:
import numpy as np

def load_wdbc(path):
    data = []
    labels = []

    with open(path, 'r') as file:
        for line in file:
            parts = line.strip().split(',')
            labels.append(1 if parts[1] == 'M' else 0)
            features = list(map(float, parts[2:]))
            data.append(features)

    return np.array(data), np.array(labels)

X, y = load_wdbc("wdbc.data")

print(X.shape)  # (569, 30)
print(y.shape)  # (569,)

def standardize(X):
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    return (X - mean) / (std + 1e-8)

X = standardize(X)

(569, 30)
(569,)


In [15]:
import numpy as np

class PCA:
    def __init__(self, n_components):
        self.n_components = n_components
        self.mean = None
        self.components = None
        self.explained_variance_ratio_ = None

    def fit(self, X):
        # 1. Center data
        self.mean = np.mean(X, axis=0)
        X_centered = X - self.mean

        # 2. Covariance matrix
        cov_matrix = np.cov(X_centered, rowvar=False)

        # 3. Eigen decomposition
        eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

        # 4. Sort eigenvalues & eigenvectors (descending)
        idx = np.argsort(eigenvalues)[::-1]
        eigenvalues = eigenvalues[idx]
        eigenvectors = eigenvectors[:, idx]

        # 5. Select top components
        self.components = eigenvectors[:, :self.n_components]

        # 6. Explained variance ratio
        total_variance = np.sum(eigenvalues)
        self.explained_variance_ratio_ = eigenvalues[:self.n_components] / total_variance

    def transform(self, X):
        X_centered = X - self.mean
        return np.dot(X_centered, self.components)

    def inverse_transform(self, X_pca):
        return np.dot(X_pca, self.components.T) + self.mean

    def reconstruction_error(self, X):
        X_pca = self.transform(X)
        X_reconstructed = self.inverse_transform(X_pca)
        return np.mean((X - X_reconstructed) ** 2)


In [16]:
pca = PCA(n_components=2)
pca.fit(X)

X_pca = pca.transform(X)
error = pca.reconstruction_error(X)

print("Explained variance ratio:", pca.explained_variance_ratio_)
print("Reconstruction error:", error)


Explained variance ratio: [0.44272051 0.1897117 ]
Reconstruction error: 0.3675674130150981


In [17]:
def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return (z > 0).astype(float)

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    s = sigmoid(z)
    return s * (1 - s)

def tanh(z):
    return np.tanh(z)

def tanh_derivative(z):
    return 1 - np.tanh(z)**2


In [18]:
class Autoencoder:
    def __init__(self, layer_sizes, activation='relu', lr=0.01, l2=0.001):
        self.layer_sizes = layer_sizes
        self.lr = lr
        self.l2 = l2

        self.activations = {
            'relu': (relu, relu_derivative),
            'sigmoid': (sigmoid, sigmoid_derivative),
            'tanh': (tanh, tanh_derivative)
        }

        self.f, self.f_prime = self.activations[activation]

        self.weights = []
        self.biases = []

        for i in range(len(layer_sizes) - 1):
            self.weights.append(
                np.random.randn(layer_sizes[i], layer_sizes[i+1]) * 0.01
            )
            self.biases.append(np.zeros((1, layer_sizes[i+1])))

    def forward(self, X):
        self.zs = []
        self.activations_cache = [X]

        for W, b in zip(self.weights, self.biases):
            z = np.dot(self.activations_cache[-1], W) + b
            self.zs.append(z)
            a = self.f(z)
            self.activations_cache.append(a)

        return self.activations_cache[-1]

    def backward(self, X):
        m = X.shape[0]
        grads_W = []
        grads_b = []

        # MSE loss derivative
        delta = (self.activations_cache[-1] - X) * self.f_prime(self.zs[-1])

        for i in reversed(range(len(self.weights))):
            dW = np.dot(self.activations_cache[i].T, delta) / m
            db = np.mean(delta, axis=0, keepdims=True)

            # L2 regularization
            dW += self.l2 * self.weights[i]

            grads_W.insert(0, dW)
            grads_b.insert(0, db)

            if i > 0:
                delta = np.dot(delta, self.weights[i].T) * self.f_prime(self.zs[i-1])

        return grads_W, grads_b

    def update(self, grads_W, grads_b):
        for i in range(len(self.weights)):
            self.weights[i] -= self.lr * grads_W[i]
            self.biases[i] -= self.lr * grads_b[i]

    def train(self, X, epochs=100, batch_size=32, decay=0.99):
        for epoch in range(epochs):
            indices = np.random.permutation(len(X))
            X_shuffled = X[indices]

            for i in range(0, len(X), batch_size):
                X_batch = X_shuffled[i:i+batch_size]
                self.forward(X_batch)
                grads_W, grads_b = self.backward(X_batch)
                self.update(grads_W, grads_b)

            self.lr *= decay  # learning rate scheduling

            if epoch % 10 == 0:
                loss = np.mean((self.forward(X) - X) ** 2)
                print(f"Epoch {epoch}, Loss: {loss:.5f}")


In [19]:
layer_sizes = [30, 20, 10, 2, 10, 20, 30]
ae = Autoencoder(layer_sizes, activation='relu', lr=0.01, l2=0.001)

ae.train(X, epochs=200, batch_size=32)


Epoch 0, Loss: 1.00000
Epoch 10, Loss: 1.00000
Epoch 20, Loss: 1.00000
Epoch 30, Loss: 1.00000
Epoch 40, Loss: 1.00000
Epoch 50, Loss: 1.00000
Epoch 60, Loss: 1.00000
Epoch 70, Loss: 1.00000
Epoch 80, Loss: 1.00000
Epoch 90, Loss: 1.00000
Epoch 100, Loss: 1.00000
Epoch 110, Loss: 1.00000
Epoch 120, Loss: 1.00000
Epoch 130, Loss: 1.00000
Epoch 140, Loss: 1.00000
Epoch 150, Loss: 1.00000
Epoch 160, Loss: 1.00000
Epoch 170, Loss: 1.00000
Epoch 180, Loss: 1.00000
Epoch 190, Loss: 1.00000
